# INF8111 - Fouille de données


## TP1 ÉTÉ 2020 - Système de recommandation

##### Membres de l'équipe:

    - Nom (Matricule) 1
    - Nom (Matricule) 2
    - Nom (Matricule) 3


## Résumé

*Stack Exchange* est un réseau de sites chacun contenant des discussions sur un sujet. Une discussion, ou *thread* en anglais, contient une question et potentiellement plusieurs réponses et commentaires. Dans ce TP, vous implémenterez un système de recommandations qui retourne les discussions en rapport à une question spécifique. Avant de soumettre une question, le site utilisera ce système de recommandations pour proposer les discussions les plus similaires afin de limiter le nombre de discussions dupliquées.


## 2 - Installation

Pour ce TP, vous aurez besoin des librairies `numpy`, `sklearn` et `scipy` (que vous avez sans doute déjà), ainsi que la librairie `nltk`, qui est une libraire utilisée pour faire du traitement du language (Natural Language Processing, NLP)

Installez les libraires en question et exécutez le code ci-dessous :

In [ ]:
# Si vous le souhaitez, vous pouvez utiliser anaconda

# pip install --user numpy
# pip install --user sklearn
# pip install --user scipy
# pip install --user nltk


#python
#import nltk
#nltk.download("punkt")

## 3 - Jeu de données

Téléchargez l'archive à l'adresse suivante: https://drive.google.com/file/d/1032N1oZkytHlHs20AXE9jPMBQCTyhb6H/view?usp=sharing

L'archive contient:
1. test.json: Ce fichier contient les nouvelles questions avec les discussions pertinentes associées.
2. threads: Ce dossier contient le code HTML des discussions. Chaque fichier HTML est nommé selon le motif **thread_id.html**.

L'image ci-dessous illustre un exemple de discussion :

![thread_img](https://i.imgur.com/fjzBQss.png)

- A : le sujet de la question
- B : le corps de la question
- C : les commentaires de la question
- D : le corps de la réponse
- E : les commentaires de la réponse



In [8]:
import os

# define the folder path that contain the data
# FOLDER_PATH = "Define folder path that contain threads folder and test.json"
FOLDER_PATH = "dataset/"
THREAD_FOLDER = os.path.join(FOLDER_PATH, 'threads')


# Load the evaluation dataset
import json


test = json.load(open(os.path.join(FOLDER_PATH, "test.json")))
relevant_threads_by_query = dict()


for (query_id, cand_id, label) in test: 
    if label == 'Irrelevant':
        continue
        
    l = relevant_threads_by_query.setdefault(query_id, [])
    l.append(cand_id)

# 4 - Web scraping

"Le *web scraping* (parfois appelé harvesting) est une technique d'extraction du contenu de sites Web, via un script ou un programme, dans le but de le transformer pour permettre son utilisation dans un autre contexte, par exemple le référencement." [Wikipedia](https://fr.wikipedia.org/wiki/Web_scraping)

## 4.1 - Question 1 (0.5 point)

Les caractères spéciaux et non-ASCII peuvent être encodés avec leur représentation HTML. Par exemple, l'apostrophe (') est encodée par **\&AMP;**. L'encodage des pages web n’est pas uniforme, seuls les caractères spéciaux et non-ASCII sont encodés avec leur représentation HTML. Cela sera rectifié en convertissant les représentations HTML en caractère UTF-8. Par exemple, **\&AMP;** sera remplacé par **'**.

Implémentez la fonction fix_encoding qui convertit les représentations HTML en caractères UTF-8.


In [ ]:
def fix_encoding(text):
    """
    Encodes the html entities in a text into UTF-8 encoding. For instance, "I&amp;m ..." => "I'm ..."
    
    :param text: string.
    :return: fixed text(sting)
    """
    raise NotImplementedError()

## 4.2 - Question 2 (3 points)

Implémentez la fonction extract_data_from_page. Cette fonction extrait le sujet de la question, son corps, ses commentaires, le corps des réponses et leurs commentaires des pages HTML. La fonction retourne un dictionnaire avec la structure suivante : *{"thread_id": int,"question":{"subject": string, "body": string, "comments": [string]}, answers: [{"body": string, "comments": [string]}]}*

**Utilisez la fonction fix_encoding pour convertir le texte. Vouz pouvez utiliser la bibliothèque [Beatiful Soup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) pour cette question. Vous devez retirer toutes HTML du texte de la question, des commentaires et des réponses. **


In [ ]:
def extract_data_from_page(pagepath):
    """
    Scrap question, answer and comments from thread page.
    
    :param pagepath: the path of thread html file.
    :return: 
        {
            "thread_id": thread id,
            "question":{
                "subject": question subject text (Area A in the figure), 
                "body": question body text (Area B in the figure), 
                "comments": list of comment texts (Area C in the figure)
                }, 
            "answers": [
                {
                    "body": answer body text (Area D in the figure),
                    "comments": list of answer texts (Area E in the figure)
                }
                ]
            }
    """
    
    # Use the fix_encoding function to fix the text encoding
    raise NotImplementedError()

## 4.3 - Extraire le texte du code HTML

In [9]:
import os
from multiprocessing import Pool, TimeoutError
from time import time
import json

# Index each thread by its id
index_path = os.path.join(THREAD_FOLDER, 'threads.json')

if os.path.isfile(index_path):
    # Load threads that webpage content were already extracted.
    thread_index = json.load(open(index_path))
else:
    # Extract webpage content

    # This can be slow (around 20 minutes). Test your code with a small sample. lxml parse is faster than html.parser
    with Pool(processes=2) as pool:
        files = [os.path.join(THREAD_FOLDER, filename) for filename in os.listdir(THREAD_FOLDER)]
        threads = pool.map(extract_data_from_page, files)
        thread_index = dict(((thread['thread_id'], thread) for thread in threads ))

    # Save preprocessed threads
    json.dump(thread_index, open(index_path,'w'))
    

## 5 - Prétraitement des données

Le prétraitement des données est une tache cruciale en fouille de données. Cette étape nettoie et transforme les données brutes dans un format qui permet leur analyse, et leur utilisation avec des algorithmes de *machine learning*. En traitement des langages (natural language processing, NLP), la *tokenization* et le *stemming* sont des étapes cruciales. De plus, vous implémenterez une étape supplémentaire pour filtrer les mots sans importance.

### 5.1 - Tokenization

Cette étape permet de séparer un texte en séquence de *tokens* (= jetons, ici des mots, symboles ou ponctuation).

Par exemple, la phrase *"It's the student's notebook."* peut être séparé en liste de tokens de cette manière: ["it", " 's", "the", "student", " 's", "notebook", "."].

**De plus, tous les tokenizers ont également le rôle de mettre le texte en minuscule.**

#### 5.1.1 - Question 3 (0.5 point) 

Implémentez la fonction suivante :

- **tokenize_space** qui tokenize le texte à partir des blancs (espace, tabulation, nouvelle ligne). Ce tokenizer est naïf.
- **tokenize_nltk** qui utilise le tokenizer par défaut de la librairie nltk (https://www.nltk.org/api/nltk.html).



In [23]:
def tokenize_space(text):

    """
    Tokenize the tokens that are separated by whitespace (space, tab, newline). 
    We consider that any tokenization was applied in the text when we use this tokenizer.
    
    For example: "hello\tworld of\nNLP" is split in ['hello', 'world', 'of', 'NLP']
    """
    # return a list of tokens
    raise NotImplementedError("")
        
def tokenize_nltk(text):
    """
    This tokenizer uses the default function of nltk package (https://www.nltk.org/api/nltk.html) to tokenize the text.
    """
    # return a list of tokens
    raise NotImplementedError("")
    
        

### 5.2 - Filtrer les tokens sans importance

#### 5.2.1 - Question 4 (1 point)

Certains tokens sont sans importance pour la comparaison, car ils apparaissent dans la majorité des discussions. Les supprimer réduit la dimension du vecteur et accélère les calculs.

Expliquez quels tokens sont sans importances pour la comparaison des discussions. Implémentez la fonction filter_word qui retire ces mots de la liste des tokens.


In [ ]:
def filter_tokens(tokens):
    raise NotImplementedError("")


### 5.3 - Stemming

La racinisation (stemming) est un procédé de transformation des flexions en leur radical ou racine. Par example, en anglais, la racinisation de "fishing", "fished" and "fish" donne "fish" (stem). 



In [25]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("english")



word1 = ["Visitors", "from", "all", "over", "the", "world", "fishes", "during", "the", "summer","."]

print([ stemmer.stem(w) for w in word1])

word2 = ['I', 'was', 'fishing',]
print([ stemmer.stem(w) for w in word2])

['visitor', 'from', 'all', 'over', 'the', 'world', 'fish', 'dure', 'the', 'summer', '.']
['i', 'was', 'fish']


#### 5.3.1 - Question 5 (1 point) 

*Expliquez comment et pourquoi le stemming est utile à note système de comparaison.*


# 6 - Data representation

## 6.1 - Bag of Words

De nombreux algorithmes demande des entrées qui soient toutes de la même taille, ce qui n'est forcément le cas pour des types de données comme les textes, qui peuvent avoir un nombre variable de mots.  

Par exemple, considérons la phrase 1, ”Board games are much better than video games” et la phrase 2, ”Monopoly is an awesome game!”. La table ci-dessous montre un exemple d'un moyen de représentation de ces deux phrases en utilisant une représentation fixe : 

|<i></i>     | an | are | ! | Monopoly | awesome | better | games | than | video | much | board | is | game |
|------------|----|-----|---|----------|---------|--------|-------|------|-------|------|-------|----|------|
| Sentence 1 | 0  | 1   | 0 | 0        | 0       | 1      | 2     | 1    | 1     | 1    | 1     | 0  | 0    |
| Sentence 2 | 1  | 0   | 1 | 1        | 1       | 0      | 0     | 0    | 0     | 0    | 0     | 1  | 1    |

Chaque colonne représente un mot du vocabulaire (de longueur 13), tandis que chaque ligne contient l'occurence des mots dans une phrase. Ainsi, la valeur 2 à la position (1,7) est due au fait que le mot *"games"* apparait deux fois dans la phrase 1. 

Ainsi, chaque ligne étant de longueur 13, on peut les utiliser comme vecteur pour représenter les phrases 1 et 2. Ainsi, c'est cette méthode que l'on appelle *Bag-of-Words* : c'est une représentation de documents par des vecteurs dont la dimension est égale à la taille du vocabulaire, et qui est construit en comptant le nombre d'occurence de chaque mot. Ainsi, chaque token est ici associé à une dimension.

### 6.1.2 - Question 6 (2.5 points)


*Implémentez le Bag-of-Words*

Pour cette question, vous ne pouvez pas utiliser de librairie Python externe comme scikit-learn, hormis si vous avez des problèmes de mémoire, vous pouvez utiliser la classe sparse.csr_matrix de scipy (https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csc_matrix.html).

In [20]:
def transform_count_bow(X):
    """
    This method preprocesses the data using the pipeline object, relates each token to a specific integer and  
    transforms the text in a vector. Vectors are weighted using the token frequencies in the sentence.

    X: document tokens. e.g: [['I','will', 'be', 'back', '.'], ['Helllo', 'world', '!'], ['If', 'you', 'insist', 'on', 'using', 'a', 'damp', 'cloth']]

    :return: vector representation of each document
    """   
    
    raise NotImplementedError("") 
    return []

## 6.2 - TF-IDF

L'utilisation de la fréquence d'apparition brute des mots, comme c'est le cas avec le bag-of-words, peut être problématique. En effet, peu de tokens auront une fréquence très élevée dans un document, et de ce fait, le poids de ces mots sera beaucoup plus grand que les autres, ce qui aura tendance à biaiser l'ensemble des poids. De plus, les mots qui apparaissent dans la plupart des documents n'aident pas à les discriminer. Par exemple, le mot "*de*" apparaît dans beaucoup de documents de la base de données, et pour autant, avoir ce mot en commun ne permet pas de conclure que des documents sont similaires. Au contraire, le mot "*génial*" est plus rare, mais les documents qui contiennent ce mot sont plus susceptibles d'être positif. TF-IDF est donc une méthode qui permet de pallier à ce problème.

TF-IDF pondère le vecteur en utilisant une fréquence de document inverse (IDF) et une fréquence de termes (TF).

TF est l'information locale sur l'importance qu'a un mot dans un document donné, tandis que IDF mesure la capacité de discrimination des mots dans un jeu de données. 

L'IDF d'un mot se calcule de la façon suivante:

\begin{equation}
	\text{idf}_i = \log\left( \frac{N}{\text{df}_i} \right),
\end{equation}

avec $N$ le nombre de documents dans la base de donnée, et $\text{df}_i$ le nombre de documents qui contiennent le mot $i$.

Le nouveau poids $w_{ij}$ d'un mot $i$ dans un document $j$ peut ensuite être calculé de la façon suivante:

\begin{equation}
	w_{ij} = \text{tf}_{ij} \times \text{idf}_i,
\end{equation}

avec $\text{tf}_{ij}$ la fréquence du mot $i$ dans le document $j$.



### 6.2.1 - Question 7 (3.5 points)

Implémentez le bag-of-words avec la pondération de TF-IDF

Pour cette question, vous ne pouvez pas utiliser de librairie Python externe comme scikit-learn, hormis si vous avez des problèmes de mémoire, vous pouvez utiliser la classe sparse.csr_matrix de scipy (https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csc_matrix.html).

In [22]:
def transform_tf_idf_bow(X):
    """
    This method preprocesses the data using the pipeline object, calculates the IDF and TF and 
    transforms the text in vectors. Vectors are weighted using TF-IDF method.

    X: document tokens. e.g: [['I','will', 'be', 'back', '.'], ['Helllo', 'world', '!'], ['If', 'you', 'insist', 'on', 'using', 'a', 'damp', 'cloth']]

    :return: vector representation of each document
    """
    raise NotImplementedError("")
    
    return []
    
        

# 7 - Système de recommandations

## 7.1 - Question 8 (1.5 points)


La *pipeline* est la séquence d'étapes de prétraitement des données qui transforme les données brutes dans un format qui permet leur analyse. Pour le problème de système de recommandations, implémentez un pipeline composé des étapes suivantes :

1. Concatène le texte de la question, des réponses et des commentaires pour chaque discussion $t$ dans le dictionnaire thread_dict.
2. Tokenize le texte.
3. Filtre les tokens sans importance.
4. Stem les tokens
5. Génère la représentation vectorielle avec transform_tf_idf_bow ou transform_count_bow.
6. Retourne les identifiants des discussions et les représentations vectorielles des discussions. 



In [ ]:
def nlp_pipeline(thread_dict, tokenization_type, vectorizer_type, enable_filter_tokens, enable_stemming):
    """
    Preprocess and vectorize the threads.
    
    thread_dict: dictionary whose keys and values are thread ids and thread objects, respectively.
    tokenization_type: two possible values "space_tokenization" and "nltk_tokenization".
                            - space_tokenization: tokenize_space function is used to tokenize.
                            - nltk_tokenization: tokenize_nltk function is used to tokenize.
                            
    vectorizer_type: two possible values "count" and "tf_idf".
                            - count: use transform_count_bow to vectorize the text
                            - tf_idf: use transform_tf_idf_bow to vectorize the text
                            
    enable_filter_tokens: enable the insignificant token removal;
    
    enable_stemming: enable stemming
    
    return: a list L with thread ids and matrix B that contains the vector of each thread. B[idx] is the fixed-length representation of L[idx].
    """
    
    raise NotImplementedError("")
    

## 7.2 - Question 9 (1.5 points)

Implémentez la fonction rank qui retourne la liste des identifiants des discussions triés par leur similarité avec la nouvelle question (query). Vous utiliserez la [cosine similarity function](https://en.wikipedia.org/wiki/Cosine_similarity) pour comparer deux discussions. Considérez la nouvelle question comme une discussion sans réponse ni commentaire.

**Retirez la nouvelle question dans la liste de recommandations**




In [ ]:
def rank(query_id, all_thread_ids, X):
    """
    Return a list of thread ids sorted by thread and query similarity. Cosine similarity is used to compare threads. 
    
    query_id: thread id 
    all_thread_ids: list of thread ids
    X: thread data representations
    
    return: ranked list of thread ids. 
    """
    
    # Compute the similarity of thread representations(vectors) using cosine similarity function
    # Sort the thread ids by the similarity
    
    raise NotImplementedError("")
    

## 7.3 - Évaluation

Vous allez tester différentes configurations du système de recommandations. Ces configurations seront comparées avec la [mean average precision (MAP) metric](https://en.wikipedia.org/wiki/Evaluation_measures_(information_retrieval)#Mean_average_precision). Plus les discussions pertinentes sont recommandées rapidement (c.-à-d. en haut de la liste), plus élevé sera le score MAP. 

Ressources supplémentaires pour comprendre MAP: [recall and precision over ranks](https://youtu.be/H7oAofuZjjE) et [MAP](https://youtu.be/pM6DJ0ZZee0).


La fonction *eval* évalue une configuration spécifique du système de recommandations.

In [24]:
from statistics import mean 


def calculate_map(x):
    res = 0.0
    n = 0.0
    
    
    for relevant_threads, ranked_list in x:
        precisions = []
               
        for k, thread_id in enumerate(ranked_list):
            if thread_id in relevant_threads:
                prec_at_k = (len(precisions) + 1)/(k+1)
                precisions.append(prec_at_k)
                
            if len(precisions) == len(relevant_threads):
                break
        res += mean(precisions)
        n += 1
    
    return res/n
            

def eval(tokenization_type, vectorizer, enable_filter_tokens, enable_stemming):
    all_thread_ids, X = nlp_pipeline(thread_index, tokenization_type, vectorizer, enable_filter_tokens, enable_stemming)
    all_thread_ids = [int(t_id) for t_id in all_thread_ids]    
    queries,relevant_threads = zip(*relevant_threads_by_query.items())
        
    def generate_rank_list(query_id):
        return rank(query_id, all_thread_ids, X)
    
    with Pool(processes=2) as pool:
        ranked_list = pool.map(generate_rank_list, queries)
        
        
    return calculate_map(zip(relevant_threads,ranked_list))
        

## 7.4 - Question 10 (5 points)

Évaluez la précision (MAP) du système de recommandations avec chacune des configurations suivantes :
1. count(BoW) + space_tokenization (sans tokenizer)
2. count(BoW) + nltk_tokenization
3. count(BoW) + nltk_tokenization + Filtrer les tokens sans importance
4. count(BoW) + nltk_tokenization + Filtrer les tokens sans importance + Stemming
5. tf_idf + nltk_tokenization
6. tf_idf + nltk_tokenization + Filtrer les tokens sans importance
7. tf_idf + nltk_tokenization + Filtrer les tokens sans importance + Stemming 

Enfin, commentez vos résultats en répondant aux questions suivantes :
- Notre système de recommandation a-t-il été influencé négativement ou positivement par les étapes de prétraitement des données ??
- Est-ce que TF-IDF est plus performant que BoW? Si oui, à votre avis, pourquoi?
